## Chapter 8: Tree Models
### Ex3: KDD 10%
Software to detect network intrusions protects a computer network
from unauthorized users, including perhaps insiders. The intrusion
detector learning task is to build a predictive model (i.e. a classifier)
capable of distinguishing between 'bad' connections, called intrusions
or attacks, and 'good' normal connections.
Read more: https://archive.ics.uci.edu/ml/datasets/KDD+Cup+1999+Data
(https://archive.ics.uci.edu/ml/datasets/KDD+Cup+1999+Data)
Requirement: We will be using a KDD dataset to try to classify a
connection as 'normal.' or others.


In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark
findspark.init()

spark-2.4.0-bin-hadoop2.7/
spark-2.4.0-bin-hadoop2.7/python/
spark-2.4.0-bin-hadoop2.7/python/setup.cfg
spark-2.4.0-bin-hadoop2.7/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/resultiterable.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/shell.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/heapq3.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/join.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/version.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/rdd.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/java_gateway.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/find_spark_home.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/_globals.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/worker.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/accumulators.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/feature.py
spark-2.4.0-bin-hadoop2.7/python/pyspark

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
%cd '/content/gdrive/My Drive/LDS9_K273_ONLINE_Đinh Viết Trung'

/content/gdrive/My Drive/LDS9_K273_ONLINE_Đinh Viết Trung


In [4]:
# import libraries
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
from pyspark.sql.functions import mean, stddev, col, log
from pyspark.sql.functions import to_date, dayofweek, to_timestamp
from pyspark.sql import types 
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType
from pyspark.sql.functions import year, month
from pyspark.sql.functions import dayofmonth, weekofyear
from pyspark.sql.functions import split, explode
from pyspark.sql.functions import coalesce, first, lit
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import Bucketizer
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.functions import datediff
from pyspark.sql.functions import when

In [5]:
from pyspark import SparkContext
sc = SparkContext()

In [6]:
spark = SparkSession(sc)

In [8]:
df = spark.read.csv('data/kddcup.data_10_percent.gz', inferSchema=True, header=False)

In [9]:
df.count()

494021

In [10]:
from pyspark.sql.functions import col

In [11]:
from pyspark.sql.functions import col

In [12]:
df.show(3)

+---+---+----+---+---+----+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-------+
|_c0|_c1| _c2|_c3|_c4| _c5|_c6|_c7|_c8|_c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|_c25|_c26|_c27|_c28|_c29|_c30|_c31|_c32|_c33|_c34|_c35|_c36|_c37|_c38|_c39|_c40|   _c41|
+---+---+----+---+---+----+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-------+
|  0|tcp|http| SF|181|5450|  0|  0|  0|  0|   0|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   8|   8| 0.0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0|   9|   9| 1.0| 0.0|0.11| 0.0| 0.0| 0.0| 0.0| 0.0|normal.|
|  0|tcp|http| SF|239| 486|  0|  0|  0|  0|   0|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   8|   8| 0.0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0|  19|  19| 1.0| 0.0|0.05

In [13]:
df.groupBy('_c41').count().show(30)

+----------------+------+
|            _c41| count|
+----------------+------+
|    warezmaster.|    20|
|          smurf.|280790|
|            pod.|   264|
|           imap.|    12|
|           nmap.|   231|
|   guess_passwd.|    53|
|        ipsweep.|  1247|
|      portsweep.|  1040|
|          satan.|  1589|
|           land.|    21|
|     loadmodule.|     9|
|      ftp_write.|     8|
|buffer_overflow.|    30|
|        rootkit.|    10|
|    warezclient.|  1020|
|       teardrop.|   979|
|           perl.|     3|
|            phf.|     4|
|       multihop.|     7|
|        neptune.|107201|
|           back.|  2203|
|            spy.|     2|
|         normal.| 97278|
+----------------+------+



In [14]:
str(df.columns)

"['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c12', '_c13', '_c14', '_c15', '_c16', '_c17', '_c18', '_c19', '_c20', '_c21', '_c22', '_c23', '_c24', '_c25', '_c26', '_c27', '_c28', '_c29', '_c30', '_c31', '_c32', '_c33', '_c34', '_c35', '_c36', '_c37', '_c38', '_c39', '_c40', '_c41']"

In [15]:
train_data,test_data = df.randomSplit([0.8,0.2])

In [16]:
test_data.groupBy('_c41').count().show(30)


+----------------+-----+
|            _c41|count|
+----------------+-----+
|    warezmaster.|    4|
|          smurf.|55877|
|            pod.|   55|
|           nmap.|   43|
|           imap.|    2|
|   guess_passwd.|   10|
|        ipsweep.|  231|
|      portsweep.|  198|
|          satan.|  341|
|           land.|    7|
|     loadmodule.|    4|
|      ftp_write.|    3|
|buffer_overflow.|    3|
|        rootkit.|    3|
|    warezclient.|  213|
|       teardrop.|  187|
|       multihop.|    3|
|        neptune.|21136|
|           back.|  425|
|            spy.|    1|
|         normal.|19310|
+----------------+-----+



In [17]:
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier, RandomForestClassifier


In [18]:
# Convert categorical strings to index values
indexer1 = StringIndexer(inputCol='_c1', outputCol='c1_idx')
indexer2 = StringIndexer(inputCol='_c2', outputCol='c2_idx')
indexer3 = StringIndexer(inputCol='_c3', outputCol='c3_idx')
indexer41 = StringIndexer(inputCol='_c41', outputCol='c41_idx')
# One-hot encode index values
onehot = OneHotEncoderEstimator(
 inputCols=['c1_idx', 'c2_idx', 'c3_idx'],
 outputCols=['c1_dummy', 'c2_dummy', 'c3_dummy']
)
# Assemble predictors into a single column
assembler = VectorAssembler(inputCols=['_c0', 'c1_dummy', 
                            'c2_dummy', 'c3_dummy', 
                            '_c4', '_c5', '_c6', 
                            '_c7', '_c8', '_c9', 
                            '_c10', '_c11', '_c12', 
                            '_c13', '_c14', 
                            '_c15', '_c16', '_c17', 
                            '_c18', '_c19', 
                            '_c20', '_c21', '_c22', 
                            '_c23', '_c24', 
                            '_c25', '_c26', '_c27', 
                            '_c28', '_c29', 
                            '_c30', '_c31', '_c32', 
                            '_c33', '_c34', 
                            '_c35', '_c36', '_c37', 
                            '_c38', '_c39', '_c40'], 
                            outputCol='features')
# A linear regression object
dtc = DecisionTreeClassifier(featuresCol='features', 
 labelCol='c41_idx', 
 predictionCol='prediction')

In [19]:
# Construct a pipeline
pipeline = Pipeline(stages=[indexer1, indexer2, indexer3, 
 indexer41, onehot, assembler, dtc])
# Train the pipeline on the training data
pipeline = pipeline.fit(train_data)
# Make predictions on the testing data
predictions = pipeline.transform(test_data)


In [20]:
# Inspect results
predictions.select("prediction", "c41_idx").show(5)


+----------+-------+
|prediction|c41_idx|
+----------+-------+
|       5.0|    5.0|
|       2.0|   10.0|
|       5.0|    5.0|
|       5.0|    5.0|
|       2.0|   10.0|
+----------+-------+
only showing top 5 rows



In [27]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Select (prediction, true label) and compute test error
acc_evaluator = MulticlassClassificationEvaluator(labelCol="c41_idx", 
 predictionCol="prediction", 
 metricName="accuracy")

In [28]:
#important: need to cast to float type, and order by prediction, else it won't wo
preds_and_labels = predictions.select(['prediction','c41_idx']).withColumn('c41_idx', col('c41_idx').cast("float")).orderBy('prediction')

In [29]:
#select only prediction and label columns
preds_and_labels = preds_and_labels.select(['prediction','c41_idx'])

In [32]:
acc_evaluator.evaluate(preds_and_labels)


0.9930587878174644

In [39]:
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Confusion matrix 
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 30)
matrix = pd.DataFrame(metrics.confusionMatrix().toArray())

In [ ]:
#matrix

In [ ]:
# With accuracy, this model is very good.

In [34]:
# New data
df_new = spark.read.csv('data/kddcup.testdata.unlabeled_10_percent.gz',
 inferSchema=True,
 header=False)


In [35]:
str(df_new.columns)


"['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c12', '_c13', '_c14', '_c15', '_c16', '_c17', '_c18', '_c19', '_c20', '_c21', '_c22', '_c23', '_c24', '_c25', '_c26', '_c27', '_c28', '_c29', '_c30', '_c31', '_c32', '_c33', '_c34', '_c35', '_c36', '_c37', '_c38', '_c39', '_c40']"

In [36]:
# Make predictions on the testing data
predictions_new = pipeline.transform(df_new)

In [37]:
predictions_new.select('features', 'prediction').show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|(113,[4,66,76,77,...|       2.0|
|(113,[4,66,76,77,...|       2.0|
|(113,[4,66,76,77,...|       2.0|
|(113,[4,66,76,77,...|       2.0|
|(113,[4,66,76,77,...|       2.0|
|(113,[4,66,76,77,...|       2.0|
|(113,[8,66,76,94,...|       2.0|
|(113,[4,66,76,77,...|       2.0|
|(113,[4,66,76,77,...|       2.0|
|(113,[2,5,66,76,7...|       2.0|
|(113,[4,66,76,77,...|       2.0|
|(113,[2,5,66,76,7...|       2.0|
|(113,[4,66,76,77,...|       2.0|
|(113,[4,66,76,77,...|       2.0|
|(113,[0,2,6,66,76...|       2.0|
|(113,[2,5,66,76,7...|       2.0|
|(113,[2,5,66,76,7...|       2.0|
|(113,[2,5,66,76,7...|       2.0|
|(113,[4,66,76,77,...|       2.0|
|(113,[4,66,76,77,...|       2.0|
+--------------------+----------+
only showing top 20 rows

